# Model Selection Enhancements

This notebook implements enhancements to the time series model selection framework (Python-only). It includes:
- RMSE_holdout (last-block holdout) selection method
- Grid search ETS fitting with an L2 penalty on parameters
- Rolling-origin cross-validation (expanding window) for model selection
- Weighted selection criterion C = w * RMSE_train + (1-w) * complexity
- Comparisons across selection methods and plotting/saving outputs

Notes and assumptions:
- Uses Python libraries: numpy, pandas, matplotlib, sklearn, statsmodels
- Attempts to load M1/M3/M4 datasets from the repository `data/` folder if present. If not found, the code demonstrates methods on a toy dataset and includes code to download datasets if needed (commented).
- For yearly series, the default holdout horizon = 6 (years). For other frequencies, horizon is inferred or 20% of training set is used.
- ETS fitting uses statsmodels' ExponentialSmoothing when possible; a custom grid-search wrapper is provided to emulate R-style parameter selection and an L2 penalty on smoothing parameters.

All outputs (CSV tables and PNG plots) are saved under `notebooks/outputs/`.


In [ ]:
# Imports and setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.simplefilter('ignore')

# Create outputs dir
OUTPUT_DIR = 'notebooks/outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)


## Utility functions
- RMSE
- split_last_block (for RMSE_holdout)
- ETS grid search with L2 penalty


In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def split_last_block(ts, holdout_pct=None, holdout_horizon=None, freq=None):
    """Split a time series (pd.Series) into train and holdout (last block).
    Provide either holdout_pct (fraction) or holdout_horizon (integer points).
    Returns: train_series, holdout_series
    """
    n = len(ts)
    if holdout_horizon is not None:
        h = int(holdout_horizon)
    elif holdout_pct is not None:
        h = max(1, int(np.ceil(n * holdout_pct)))
    else:
        # default 20%
        h = max(1, int(np.ceil(n * 0.2)))
    if h >= n:
        raise ValueError('Holdout horizon >= series length')
    train = ts.iloc[:-h]
    holdout = ts.iloc[-h:]
    return train, holdout

def ets_grid_search(ts_train, ts_holdout, seasonal_periods=None, param_grid=None, l2_penalty=0.01, seasonal=None, trend_options=[None,'add','mul'], damped_options=[False, True], maxiter=100):
    """Grid search over ETS model configurations and smoothing params.
    Uses statsmodels ExponentialSmoothing with optimized=False for explicit smoothing params.
    param_grid: dict with keys 'alpha','beta','gamma' ranges (iterables)
    Returns best_model_info dict and DataFrame of results.
    """
    results = []
    # Default param ranges if not provided
    if param_grid is None:
        param_grid = {
            'alpha': np.linspace(0.01, 0.99, 10),
            'beta': np.linspace(0.0, 0.5, 6),
            'gamma': np.linspace(0.0, 0.5, 6)
        }
    # For yearly data, seasonal is typically None; seasonal_periods None means no seasonality
    for trend in trend_options:
        for damped in damped_options:
            # If seasonality not desired skip gamma loops
            seasonal_range = param_grid.get('gamma', [0.0]) if seasonal is not None else [0.0]
            for alpha in param_grid['alpha']:
                for beta in param_grid['beta']:
                    for gamma in seasonal_range:
                        try:
                            model = ExponentialSmoothing(ts_train, trend=trend, damped_trend=damped, seasonal=seasonal, seasonal_periods=seasonal_periods)
                            # Fit with provided smoothing params (do not optimize inside fit)
                            fit_kwargs = dict(smoothing_level=alpha)
                            if trend is not None:
                                fit_kwargs['smoothing_slope'] = beta
                                # statsmodels expects damping_slope only when optimized; for fixed smoothing params we may leave it
                            if seasonal is not None:
                                fit_kwargs['smoothing_seasonal'] = gamma
                            fitted = model.fit(optimized=False, use_brute=False, start_params=None, **fit_kwargs)
                            # Forecast for holdout horizon
                            h = len(ts_holdout)
                            fc = fitted.forecast(h)
                            score = rmse(ts_holdout.values, fc)
                            # L2 penalty on parameters
                            l2 = l2_penalty * (alpha**2 + beta**2 + gamma**2)
                            penalized_score = score + l2
                            results.append({
                                'trend': trend,
                                'damped': damped,
                                'alpha': float(alpha),
                                'beta': float(beta),
                                'gamma': float(gamma),
                                'rmse_holdout': float(score),
                                'penalized_rmse': float(penalized_score)
                            })
                        except Exception as e:
                            # skip configurations that fail
                            # print('skip', trend, damped, alpha, beta, gamma, e)
                            continue
    df = pd.DataFrame(results)
    if df.empty:
        raise RuntimeError('No successful ETS fits during grid search')
    best_row = df.loc[df['rmse_holdout'].idxmin()]
    return best_row.to_dict(), df


## Toy dataset demo
Generate a toy daily series (100 points, 2023-01-01 to 2023-04-10) and run the selection methods to demonstrate outputs.


In [ ]:
# Toy dataset (100 daily samples)
rng = pd.date_range('2023-01-01', periods=100, freq='D')
np.random.seed(0)
toy_values = 10 + np.linspace(0, 2, 100) + np.sin(np.linspace(0, 6.28, 100)) + np.random.normal(0, 0.5, 100)
toy_series = pd.Series(toy_values, index=rng).rename('toy')

# Split last block: use 20% holdout
train, holdout = split_last_block(toy_series, holdout_pct=0.2)
print('toy length', len(toy_series), 'train', len(train), 'holdout', len(holdout))

# Simple ETS grid for toy dataset (seasonality daily -> weekly?)
seasonal = 'add'  # toy has weekly-ish pattern but keep simple
seasonal_periods = 7
param_grid = {
    'alpha': np.linspace(0.01, 0.9, 8),
    'beta': np.linspace(0.0, 0.5, 5),
    'gamma': np.linspace(0.0, 0.5, 5)
}
best_info, grid_df = ets_grid_search(train, holdout, seasonal_periods=seasonal_periods, param_grid=param_grid, l2_penalty=0.01, seasonal=seasonal, trend_options=[None,'add'], damped_options=[False])
best_info


## Rolling-origin cross-validation (expanding window)
Implements expanding training windows and fixed test horizon. Returns average RMSE across folds for a given ETS configuration.


In [ ]:
def rolling_origin_cv(ts, initial_train_size, horizon, step=1, trend=None, seasonal=None, seasonal_periods=None, alpha=None, beta=None, gamma=None, l2_penalty=0.0):
    """Perform rolling-origin CV with expanding window.
    ts: pd.Series
    initial_train_size: number of points to start with
    horizon: fixed forecast horizon
    step: step size for advancing origin
    Returns mean RMSE across folds and list of fold RMSEs
    """
    n = len(ts)
    fold_rmse = []
    start = initial_train_size
    while start + horizon <= n:
        train = ts.iloc[:start]
        test = ts.iloc[start:start+horizon]
        try:
            model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods)
            fit_kwargs = {}
            if alpha is not None:
                fit_kwargs['smoothing_level'] = alpha
            if beta is not None and trend is not None:
                fit_kwargs['smoothing_slope'] = beta
            if gamma is not None and seasonal is not None:
                fit_kwargs['smoothing_seasonal'] = gamma
            fitted = model.fit(optimized=False, **fit_kwargs)
            fc = fitted.forecast(horizon)
            s = rmse(test.values, fc)
            penalized = s + l2_penalty * sum([(p or 0.0)**2 for p in [alpha, beta, gamma]])
            fold_rmse.append(penalized)
        except Exception:
            # if fit fails, append a large penalty
            fold_rmse.append(np.nan)
        start += step
    fold_rmse = [x for x in fold_rmse if not np.isnan(x)]
    if not fold_rmse:
        return np.nan, []
    return float(np.mean(fold_rmse)), fold_rmse

# Example of rolling CV on toy series
mean_rmse, folds = rolling_origin_cv(toy_series, initial_train_size=50, horizon=5, step=5, trend='add', seasonal='add', seasonal_periods=7, alpha=0.3, beta=0.05, gamma=0.05)
mean_rmse, folds[:5]


## Weighted criterion C = w * RMSE_train + (1-w) * complexity
Complexity is number of ETS parameters used: alpha (1), beta (if trend) (+1), gamma (if seasonal) (+1). We'll test weights w in {0.3,0.5,0.7}.


In [ ]:
def complexity_of_config(trend=None, seasonal=None):
    c = 1  # alpha always present
    if trend is not None:
        c += 1
    if seasonal is not None:
        c += 1
    return c

def select_by_weighted_criterion(ts_train, ts_holdout, configs, w=0.5, l2_penalty=0.0):
    """configs: iterable of dicts with keys trend, seasonal, alpha,beta,gamma,seasonal_periods,damped
    Returns best config by criterion C.
    """
    rows = []
    for cfg in configs:
        try:
            model = ExponentialSmoothing(ts_train, trend=cfg.get('trend'), damped_trend=cfg.get('damped', False), seasonal=cfg.get('seasonal'), seasonal_periods=cfg.get('seasonal_periods'))
            fit_kwargs = {}
            if 'alpha' in cfg:
                fit_kwargs['smoothing_level'] = cfg['alpha']
            if 'beta' in cfg and cfg.get('trend') is not None:
                fit_kwargs['smoothing_slope'] = cfg['beta']
            if 'gamma' in cfg and cfg.get('seasonal') is not None:
                fit_kwargs['smoothing_seasonal'] = cfg['gamma']
            fitted = model.fit(optimized=False, **fit_kwargs)
            h = len(ts_holdout)
            fc = fitted.forecast(h)
            rmse_hold = rmse(ts_holdout.values, fc)
            rmse_train = rmse(ts_train.values, fitted.fittedvalues)
            complexity = complexity_of_config(cfg.get('trend'), cfg.get('seasonal'))
            C = w * rmse_train + (1-w) * complexity
            rows.append({**cfg, 'rmse_holdout':rmse_hold, 'rmse_train':rmse_train, 'complexity':complexity, 'C':C})
        except Exception:
            continue
    df = pd.DataFrame(rows)
    if df.empty:
        return None, df
    best = df.loc[df['C'].idxmin()].to_dict()
    return best, df

# Example configs (small) for toy data
configs = [
    {'trend':None, 'seasonal':'add', 'seasonal_periods':7, 'alpha':0.2, 'beta':0.0, 'gamma':0.1, 'damped':False},
    {'trend':'add', 'seasonal':'add', 'seasonal_periods':7, 'alpha':0.3, 'beta':0.05, 'gamma':0.05, 'damped':False},
    {'trend':'add', 'seasonal':None, 'alpha':0.4, 'beta':0.1, 'gamma':0.0, 'damped':False}
]
best_w, dfw = select_by_weighted_criterion(train, holdout, configs, w=0.5)
best_w


## Save demonstration outputs (toy) — RMSE comparison table & plot


In [ ]:
# For demonstration create a small comparison table for toy series using several selection criteria
comparison = []

# 1) AICc / AIC - approximate via fitted model.aic (statsmodels provides aic)
try:
    m_full = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=7).fit(optimized=True)
    aic_full = m_full.aic if hasattr(m_full, 'aic') else np.nan
except Exception:
    aic_full = np.nan

# 2) RMSE_train (choose model with lowest in-sample RMSE) - we evaluate three candidate configs
candidate_configs = configs
best_by_train = None
min_train_rmse = np.inf
for cfg in candidate_configs:
    try:
        model = ExponentialSmoothing(train, trend=cfg.get('trend'), seasonal=cfg.get('seasonal'), seasonal_periods=cfg.get('seasonal_periods'))
        fit_kwargs = {}
        if 'alpha' in cfg:
            fit_kwargs['smoothing_level'] = cfg['alpha']
        if 'beta' in cfg and cfg.get('trend') is not None:
            fit_kwargs['smoothing_slope'] = cfg['beta']
        fitted = model.fit(optimized=False, **fit_kwargs)
        train_rmse = rmse(train.values, fitted.fittedvalues)
        test_rmse = rmse(holdout.values, fitted.forecast(len(holdout)))
        if train_rmse < min_train_rmse:
            min_train_rmse = train_rmse
            best_by_train = {'cfg':cfg, 'train_rmse':train_rmse, 'test_rmse':test_rmse}
    except Exception:
        continue

# 3) RMSE_holdout from previous grid search (best_info)
best_holdout = best_info

# 4) Weighted criterion best (w=0.5)
best_weighted = best_w

comparison = pd.DataFrame([
    {'method':'AICc/AIC', 'rmse_test': (aic_full if np.isfinite(aic_full) else np.nan)},
    {'method':'RMSE_train', 'rmse_test': best_by_train['test_rmse'] if best_by_train is not None else np.nan},
    {'method':'RMSE_holdout', 'rmse_test': best_holdout.get('rmse_holdout', np.nan)},
    {'method':'Weighted_w0.5', 'rmse_test': best_weighted.get('rmse_holdout') if best_weighted is not None else np.nan}
])
comparison.to_csv(os.path.join(OUTPUT_DIR, 'rmse_comparison_table.csv'), index=False)

ax = comparison.set_index('method').plot(kind='bar', legend=False, ylabel='RMSE / AIC(approx)')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'rmse_comparison_plot.png'), dpi=200)
comparison


## Next steps and placeholders for working on M1/M3/M4 datasets
The repository may include M1, M3, M4 files under `data/`. If available, the next cells should:
- Load yearly series subsets from M1/M3/M4
- For each series, run model selection methods (AICC/BIC/RMSE_train/RMSE_holdout/rolling CV/weighted)
- Aggregate results into CSVs and plots as specified in the task.

Because dataset file formats vary (e.g., nested lists in .txt, .csv of series), the notebook provides utility code below to try to detect and load series files. If files are not present, the notebook will continue with toy demo only and documents that dataset loading is required.


In [ ]:
def find_series_files(data_dir='data'):
    """Look for likely M1/M3/M4 files in repository data folder."""
    if not os.path.exists(data_dir):
        return []
    files = []
    for root, dirs, filenames in os.walk(data_dir):
        for fn in filenames:
            if fn.lower().endswith(('.csv', '.txt', '.tsv', '.dat')):
                files.append(os.path.join(root, fn))
    return files

files = find_series_files('data')
files[:10]


## Documentation cell — CV definition and Fotios Fig 2 reproduction notes
This notebook assumes CV in Fotios' paper refers to rolling-origin cross-validation with expanding windows and a fixed horizon (e.g., 6 years for yearly series). The notebook provides a rolling-origin implementation and a plan to reproduce Fig 2 by comparing AIC-based selection vs CV-based selection across a subset of series, plotting median RMSE_test or similar metrics. If data or exact R supplementary code differences exist, the notebook documents them and provides a reproducible Python alternative.


## Next actions (to be implemented by script `scripts/model_selection_v3.py`)
- Wrap the selection methods into reusable functions in scripts/model_selection_v3.py
- Batch process the toy and M1/M3/M4 series, create CSV result tables, and save plots.

This notebook demonstrates the algorithms and saves example outputs for the toy dataset. Proceed to create the script that will run the full experiments and produce all deliverables when datasets are available.
